In [2]:
import torch
import pandas as pd
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from MyLSTM import MyLSTM
import torch.nn as nn

/Users/xinghe/.pyenv/versions/3.9.6/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)

mps


In [4]:
def generate_recipe(model, tokenizer, ingredients, max_length=128):
    model.eval()  # Switch to evaluation mode

    # Process ingredients into a single string if list, and tokenize
    ingredients = ' '.join(ingredients) if isinstance(ingredients, list) else ingredients
    input_ids = tokenizer.encode(ingredients, return_tensors='pt')

    # Prepare initial hidden state and cell state for LSTM
    # Initialize for all layers: num_layers, batch_size, hidden_dim
    h, c = [torch.zeros(model.lstm.num_layers, 1, model.lstm.hidden_size) for _ in range(2)]  # Adjusted for all layers

    # Prepare the sequence for generated ids, starting with the initial input
    generated_ids = input_ids

    with torch.no_grad():
        for _ in range(max_length):
            output, (h, c) = model.lstm(model.embedding(generated_ids[:, -1:]), (h, c))
            output = model.fc(output.squeeze(1))  # Assuming model.fc is the final fully connected layer
            next_token_id = torch.argmax(output, dim=1).unsqueeze(1)

            generated_ids = torch.cat((generated_ids, next_token_id), dim=1)

            # Check if the next token is the separator token, indicating the end
            if next_token_id.item() == tokenizer.sep_token_id:
                break

    # Decode the sequence of token ids to a string
    generated_text = tokenizer.decode(generated_ids.squeeze(), skip_special_tokens=True)
    print('Generated IDs:', generated_ids.squeeze())
    return generated_text

In [5]:
data = pd.read_excel("recipes_20000.xlsx")
training_data = data.head(1000)
max_length = 128

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
vocab_size = tokenizer.vocab_size

input_ids = []
output_ids = []

for index, row in training_data.iterrows():
    # string (list shape) to string(space split)
    ingredients = (' ').join(eval(row['ingredients']))
    steps = (' ').join(eval(row['steps']))

    # tokenized input and output
    input_tokens = tokenizer(ingredients, return_tensors='pt', padding='max_length', truncation=True, max_length=max_length)
    output_tokens = tokenizer(steps, return_tensors='pt', padding='max_length', truncation=True, max_length=max_length)

    # add label
    input_ids.append(input_tokens['input_ids'])
    output_ids.append(output_tokens['input_ids'])

    # convert to torch
    input_ids_tensor = torch.stack(input_ids)
    output_ids_tensor = torch.stack(output_ids)

In [6]:
# embedding_dim = 128
# hidden_dim = 256
# num_layers = 5
# dropout_rate = 0.1
# batch_size = 64
# epochs = 5

# # Model
# model_lstm = EnhancedLSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate)

# dataset = TensorDataset(input_ids_tensor, output_ids_tensor)
# train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # Optimizer
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model_lstm.parameters(), lr=0.001)

# # Training
# model_lstm.train()
# for epoch in range(epochs):
#     total_loss = 0
#     for inputs, targets in train_loader:
#         inputs = inputs.squeeze(1)  # Remove the singleton dimension if present
#         targets = targets.squeeze(1)  # Similarly, ensure targets are correctly shaped
#         optimizer.zero_grad()

#         outputs = model_lstm(inputs) 
#         loss = criterion(outputs.view(-1, vocab_size), targets.view(-1))
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()

#     average_loss = total_loss / len(train_loader)
#     print(f'Epoch {epoch+1}, Average Loss: {average_loss:.4f}')


In [55]:
# Training for MyLSTM
embedding_dim = 128
hidden_dim = 256
num_layers = 5
dropout_rate = 0.1
batch_size = 64
epochs = 5

# Model
model_mylstm = MyLSTM(units=vocab_size, input_size=embedding_dim)

dataset = TensorDataset(input_ids_tensor, output_ids_tensor)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model_mylstm.parameters(), lr=0.001)

In [ ]:
model_mylstm.train()
for epoch in range(epochs):
    total_loss = 0
    for inputs, targets in train_loader:
        inputs = inputs.float()  
        targets = targets.squeeze(1) 

        optimizer.zero_grad()
        outputs, _ = model_mylstm(inputs) 
        outputs = outputs[-1]  

        loss = criterion(outputs.view(-1, vocab_size), targets.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Average Loss: {average_loss:.4f}')

In [10]:
# Torch.nn.LSTM
class RecipeLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate=0.1):
        super(RecipeLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers, dropout=dropout_rate, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        embedded = self.embedding(x)
        lstm_out, _ = self.lstm(embedded)
        final_output = self.fc(lstm_out)
        return final_output

In [11]:
# Training for RecipeLSTM
embedding_dim = 128
hidden_dim = 256
num_layers = 5
dropout_rate = 0.1
batch_size = 64
epochs = 5

# Model
model = RecipeLSTM(vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate)

# Data
dataset = TensorDataset(input_ids_tensor, output_ids_tensor)
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Optimization
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [12]:
# Training Loop
model.train()
for epoch in range(epochs):
    total_loss = 0
    for inputs, targets in train_loader:
        inputs = inputs.squeeze(1)  # Remove the singleton dimension if present
        targets = targets.squeeze(1)  # Similarly, ensure targets are correctly shaped

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.view(-1, vocab_size), targets.view(-1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}, Average Loss: {average_loss:.4f}')


Epoch 1, Average Loss: 7.9862
Epoch 2, Average Loss: 5.4272
Epoch 3, Average Loss: 5.2742
Epoch 4, Average Loss: 5.2556
Epoch 5, Average Loss: 5.2321


In [13]:
# test
ingredients_list = ['onion', 'red bell pepper', 'garlic cloves']

# generate
recipe_text = generate_recipe(model, tokenizer, ingredients_list)
# print(recipe_text)


Generated IDs: tensor([  101, 20949,  2417,  4330, 11565, 20548, 18856, 21818,  2015,   102,
          101,   101,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,

In [14]:
recipe_text

'onion red bell pepper garlic cloves'